In [1]:
# Model without CRF

from keras.utils.np_utils import to_categorical
#from tflearn.data_utils import to_categorical

Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [5]:
import pandas as pd
import numpy as np
from keras.models import load_model


data = pd.read_csv('nptdata.csv', encoding='latin1', sep = ',')
data = data.fillna(method="ffill")
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(data["Tag"].values))
n_tags = len(tags)

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [6]:
import keras

In [7]:
import tensorflow as tf

In [8]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=100, validation_split=0.1, verbose=1)

#model.save('mmmodel.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#modelkk = load_model('mmmodel.h5')

Train on 1711 samples, validate on 191 samples
Epoch 1/100
1711/1711 [==============================] - 12s - loss: 0.4295 - acc: 0.9333 - val_loss: 0.2487 - val_acc: 0.9485
Epoch 2/100
1711/1711 [==============================] - 10s - loss: 0.2198 - acc: 0.9493 - val_loss: 0.2034 - val_acc: 0.9486
Epoch 3/100
1711/1711 [==============================] - 11s - loss: 0.1601 - acc: 0.9557 - val_loss: 0.1429 - val_acc: 0.9617
Epoch 4/100
1711/1711 [==============================] - 11s - loss: 0.1158 - acc: 0.9658 - val_loss: 0.1166 - val_acc: 0.9664
Epoch 5/100
1711/1711 [==============================] - 12s - loss: 0.0932 - acc: 0.9733 - val_loss: 0.0956 - val_acc: 0.9743
Epoch 6/100
1711/1711 [==============================] - 11s - loss: 0.0755 - acc: 0.9790 - val_loss: 0.0819 - val_acc: 0.9791
Epoch 7/100
1711/1711 [==============================] - 11s - loss: 0.0595 - acc: 0.9836 - val_loss: 0.0696 - val_acc: 0.9828
Epoch 8/100
1711/1711 [==============================] - 11s - l

1711/1711 [==============================] - 12s - loss: 2.8920e-04 - acc: 0.9999 - val_loss: 0.0273 - val_acc: 0.9962
Epoch 64/100
1711/1711 [==============================] - 11s - loss: 4.9407e-04 - acc: 0.9998 - val_loss: 0.0270 - val_acc: 0.9963
Epoch 65/100
1711/1711 [==============================] - 11s - loss: 3.8344e-04 - acc: 0.9999 - val_loss: 0.0257 - val_acc: 0.9963c: 0. - ETA: 7s - loss: 2.8343e-04 - acc: 0.99 - ETA
Epoch 66/100
1711/1711 [==============================] - 12s - loss: 3.3860e-04 - acc: 0.9999 - val_loss: 0.0315 - val_acc: 0.9961
Epoch 67/100
1711/1711 [==============================] - 13s - loss: 3.9437e-04 - acc: 0.9999 - val_loss: 0.0261 - val_acc: 0.9964
Epoch 68/100
1711/1711 [==============================] - 12s - loss: 4.0611e-04 - acc: 0.9999 - val_loss: 0.0255 - val_acc: 0.9963
Epoch 69/100
1711/1711 [==============================] - 13s - loss: 3.7658e-04 - acc: 0.9999 - val_loss: 0.0282 - val_acc: 0.9959
Epoch 70/100
1711/1711 [=============

In [9]:
test_sentence = "An individual wiping command should be implemented by setting the level of WW and INT-line from HIGH to LOW for the parameterized time p_t_einzelwischhub body control module".split(" ")

In [10]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [11]:
def newpredict(tt):
    result = []
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in tt]],
                            padding="post", value=0, maxlen=max_len)
    p = model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(tt, p[0]):
        result.append((w, tags[pred]))
    return result

In [12]:
newpredict(test_sentence)

[('An', 'O'),
 ('individual', 'O'),
 ('wiping', 'O'),
 ('command', 'O'),
 ('should', 'O'),
 ('be', 'O'),
 ('implemented', 'O'),
 ('by', 'O'),
 ('setting', 'O'),
 ('the', 'O'),
 ('level', 'O'),
 ('of', 'O'),
 ('WW', 'O'),
 ('and', 'O'),
 ('INT-line', 'O'),
 ('from', 'O'),
 ('HIGH', 'O'),
 ('to', 'O'),
 ('LOW', 'O'),
 ('for', 'O'),
 ('the', 'O'),
 ('parameterized', 'O'),
 ('time', 'O'),
 ('p_t_einzelwischhub', 'O'),
 ('body', 'O'),
 ('control', 'O'),
 ('module', 'O')]

In [13]:
[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

In [14]:
scores = model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1888/1902 [============================>.] - ETA: 0sacc: 99.95%


In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("modelpp6.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelpp6.h5")
print("Saved model to disk")

Saved model to disk


In [18]:
from keras.models import model_from_json
# load json and create model
json_file = open('modelpp6.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelpp6.h5")
print("Loaded model from disk")

Loaded model from disk


In [19]:
import os
os.getcwd()

'C:\\Users\\jmo4cob\\1_ARISE\\8_Workshop\\NER_workshop'

In [20]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [21]:
score = loaded_model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

1888/1902 [============================>.] - ETA: 0sacc: 99.95%


In [22]:
import numpy as np

# Save
np.save('word2idx.npy', word2idx) 

# Load
#word2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\word2idx.npy').item()


# Save
np.save('tag2idx.npy', tag2idx) 

# Load
#tag2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\tag2idx.npy').item()

In [24]:
def bilstmfinal(test_sentence):
    import pandas as pd
    import numpy as np
    from keras.models import model_from_json
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Model, Input
    from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import numpy
    numpy.random.seed(7)
    word2idx = np.load('word2idx.npy').item()
    tag2idx = np.load('tag2idx.npy').item()
    # load json and create model
    json_file = open('modelpp6.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("modelpp6.h5")
    loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    test_sentence = test_sentence.split(" ")
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=50)
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    pls = []
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(test_sentence, p[0]):
        pls.append(pred)
    revsubs = { v:k for k,v in tag2idx.items()}
    entit = [revsubs.get(item,item) for item in pls]
    final = list(zip(test_sentence,entit))
    return final

In [25]:
s = "The system shall allow a registered visitor to load a custom filter."

In [26]:
bilstmfinal(s)

[('The', 'O'),
 ('system', 'O'),
 ('shall', 'O'),
 ('allow', 'O'),
 ('a', 'O'),
 ('registered', 'actor'),
 ('visitor', 'actor'),
 ('to', 'O'),
 ('load', 'action'),
 ('a', 'O'),
 ('custom', 'attribute'),
 ('filter.', 'O')]

In [84]:
# def lstm_model():
#     import pandas as pd
#     import numpy as np
#     from keras.models import load_model


#     data = pd.read_csv('C:\\users\\sharath\\postags_v1.csv', encoding='latin1', sep = ',')
#     data = data.fillna(method="ffill")
#     words = list(set(data["Word"].values))
#     words.append("ENDPAD")
#     n_words = len(words)
#     tags = list(set(data["Tag"].values))
#     n_tags = len(tags)

#     class SentenceGetter(object):

#         def __init__(self, data):
#             self.n_sent = 1
#             self.data = data
#             self.empty = False
#             agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
#                                                                s["POS"].values.tolist(),
#                                                                s["Tag"].values.tolist())]
#             self.grouped = self.data.groupby("Sentence #").apply(agg_func)
#             self.sentences = [s for s in self.grouped]

#         def get_next(self):
#             try:
#                 s = self.grouped["Sentence: {}".format(self.n_sent)]
#                 self.n_sent += 1
#                 return s
#             except:
#                 return None

#     getter = SentenceGetter(data)
#     sent = getter.get_next()
#     sentences = getter.sentences
#     max_len = 50
#     word2idx = {w: i for i, w in enumerate(words)}
#     tag2idx = {t: i for i, t in enumerate(tags)}
#     from keras.preprocessing.sequence import pad_sequences
#     X = [[word2idx[w[0]] for w in s] for s in sentences]
#     X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
#     y = [[tag2idx[w[2]] for w in s] for s in sentences]
#     y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#     from keras.utils import to_categorical
#     y = [to_categorical(i, num_classes=n_tags) for i in y]
#     from sklearn.model_selection import train_test_split
#     X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
#     from keras.models import Model, Input
#     from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
#     input = Input(shape=(max_len,))
#     model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
#     model = Dropout(0.1)(model)
#     model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
#     out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
#     model = Model(input, out)
#     model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
#     history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=10, validation_split=0.1, verbose=1)
#     history.model.save('m3odel.h5')  # creates a HDF5 file 'my_model.h5'
#     del model  # deletes the existing model
#     # returns a compiled model
#     # identical to the previous one
#     modelkk = load_model('m3odel.h5')
#     return modelkk

In [2]:
import tensorflow as tf

In [4]:
sess

In [8]:
tt = "The system shall allow the data manager to search occurrence resources by keywords"

In [9]:
tt = "The system shall allow the data manager to edit the metadata of an institution resource"